In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import requests
import json
import numpy as np
import time
from selenium.webdriver.common.keys import Keys

In [2]:
URL = "www.video_url.com"

In [ ]:
driver = webdriver.Chrome(executable_path=r"D:\\chromedriver.exe")

In [4]:
driver.get(URL)

In [ ]:
User = driver.find_element_by_id("user_email")
User.send_keys('account_name')
PW = driver.find_element_by_id("user_password")
PW.send_keys('password')
button = driver.find_element_by_name('commit')
button.click()

In [6]:
driver.maximize_window()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [25]:
load_btn = driver.find_element_by_id('media_stats_load_more')
count = 0 
while True:
    time.sleep(3)
    if load_btn.is_displayed() and load_btn.is_enabled():
        load_btn.click()
        count+= 1
        continue
    else:
        print('Load More Ends!')
        break
count

203

In [26]:
page_source = driver.page_source

In [27]:
soup = BeautifulSoup(page_source, 'html.parser')

In [48]:
df_viewerdemo = pd.DataFrame(columns=['City', 'State', 'Country', 'Device', 'OS', 'Browser', 'Day', 'Local Time', 'PecentViewed','IP', 'Heatmap', 'ID'])

In [53]:
df_viewerdemo1 = pd.DataFrame(columns=['City', 'State', 'Country', 'Device', 'OS', 'Browser', 'Day', 'Local Time', 'PecentViewed','IP', 'Heatmap', 'ID'])

In [68]:
def datetime(i):
    datetime = viewer_info[i].find('time')['datetime'].strip('Z').split('T')
    ID = viewer_info[i].get('id')
    Day = datetime[0]
    Time = datetime[1]
    return Day, Time, ID

('2022-04-27', '03:53:47', 'stat-event-ERsjrHrIxOWHQ7y47E_YlQ')

In [82]:
def media(i):
    media = viewer_info[i].find('div', class_='media-body stat-detail').text.strip('   https://www.saenchaimuaythai.com/\n').split()
    if len(media) > 1:
        Device = media[0]
        OS = media[2]
        Browser = media[4] + media[5] if media[4][0] == 'I' else media[4]
        IP = media[6] if media[4][0] == 'I' else media[5]
        return Device, OS, Browser, IP
    else:
        IP = media[0]
        return '', '', '', IP

In [83]:
def location(i):
    location = viewer_info[i].find('div', class_="media-event-location media-body media-middle").text.strip().split(',')
    City = location[0]
    State = location[1]
    Country = location[2]
    return City, State, Country

In [2]:
def heatmap(i):
    Heatmap_url = viewer_info[i].find('iframe')['src']
    r = requests.get(Heatmap_url)
    soup = BeautifulSoup(r.content, 'html5lib')
    htmap = soup.find_all('script')[4]
    timelineData = htmap.string.strip().split()[-1].rstrip(');')
    heatmap_dict = json.loads(timelineData)
    PecentViewed = heatmap_dict['percentViewed']  
    return Heatmap_url, PecentViewed, heatmap_dict

In [4]:
df = pd.read_excel('Demographic.xlsx')

In [3]:
def heatmap_data(i):
    Heatmap_url = df['Heatmap'][i]
    r = requests.get(Heatmap_url)
    soup = BeautifulSoup(r.content, 'html5lib')
    htmap = soup.find_all('script')[4]
    timelineData = htmap.string.strip().split()[-1].rstrip(');')
    heatmap_dict = json.loads(timelineData)
    timestamp = heatmap_dict['timelineData']
    return timestamp


In [ ]:
lst = []
for url in range(0, len(df['Heatmap'])):
    timedata = heatmap_data(url)
    for i in range(1, len(timedata)-1, 2):
        items = [timedata[i][0],timedata[i+1][0], timedata[i][1]]
        lst.append(items)

In [14]:
df_heatmap = pd.DataFrame(lst)
df_heatmap.columns = ['Start', 'End', 'Re-played']
df_heatmap

,Start,End,Re-played
0,0.000,6.234,2
1,6.234,42.900,1
2,42.900,58.054,2
3,58.054,192.000,1
4,192.000,192.198,2
...,...,...,...
9882,610.600,610.660,1
9883,610.660,617.600,0
9884,617.600,619.325,1
9885,0.100,255.529,1


In [4]:
replayed = pd.read_excel('replayed.xlsx')

In [ ]:
viewer demographic
viewer_info = soup.find_all('div', class_='media-event')

for i in range(758, len(viewer_info)):
    City = location(i)[0]
    State = location(i)[1]
    Country = location(i)[2]
    Device = media(i)[0]
    OS = media(i)[1]
    Browser = media(i)[2]
    IP = media(i)[3]
    Day = datetime(i)[0]
    Time = datetime(i)[1]
    ID = datetime(i)[2]
    Heatmap_url = heatmap(i)[0]
    PecentViewed = heatmap(i)[1]
    df_viewerdemo1.loc[i+1] = [City, State, Country, Device, OS, Browser, Day, Time, PecentViewed, IP, Heatmap_url, ID]
    

In [51]:
df_viewerdemo.to_excel('Demographic.xlsx')

In [20]:
#close everything
drive.quit()